<a href="https://colab.research.google.com/github/afairley19/nlp_colab/blob/master/biden_trump_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"] 
! java -version
# Install pyspark
! pip install --ignore-installed pyspark==2.4.4
# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.6.3

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)
     |████████████████████████████████| 215.7MB 61kB/s 
     |████████████████████████████████| 204kB 42.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130389 sha256=5a557e7f9962ad3b0d4e0f05b110f40a800396d75f954e068f7e6673c13abf89
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 133kB 9.3MB/s 


In [2]:
! sudo update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [3]:
from pyspark.sql import SparkSession 
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline 
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version()) 
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.6.3
Apache Spark version:  2.4.4


In [4]:
import pickle
import cloudpickle as cp
lst_donald_trump_tweets=[]
lst_joe_biden_tweets=[]

from urllib.request import urlopen
lst_donald_trump_tweets = cp.load(urlopen("https://raw.githubusercontent.com/gomachinelearning/Blogs/master/DonaldTrumpTweets.pickle"))
lst_joe_biden_tweets = cp.load(urlopen("https://raw.githubusercontent.com/gomachinelearning/Blogs/master/JoeBidenTweets.pickle"))

In [9]:
import pandas as pd
dt_df = spark.createDataFrame(pd.DataFrame({"trump_tweets":lst_donald_trump_tweets}))
jb_df = spark.createDataFrame(pd.DataFrame({"biden_tweets":lst_joe_biden_tweets}))

In [12]:
dt_df.printSchema()

dt_df.count()

root
 |-- trump_tweets: string (nullable = true)



3237

In [13]:
jb_df.printSchema()

jb_df.count()

root
 |-- biden_tweets: string (nullable = true)



3230

In [14]:
documentAssembler = DocumentAssembler()\
.setInputCol("trump_tweets")\
.setOutputCol("trump_document")

use = UniversalSentenceEncoder.pretrained(lang="en")\
.setInputCols(["trump_document"])\
.setOutputCol("dt_sentence_embeddings")

model_name = 'classifierdl_use_fakenews'
document_classifier = ClassifierDLModel.pretrained(model_name)\
.setInputCols(['trump_document', 'dt_sentence_embeddings']).setOutputCol("dt_prediction")

nlp_Pipeline = Pipeline( stages=[documentAssembler,
            use,
            document_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


In [17]:
jb_documentAssembler = DocumentAssembler()\
.setInputCol("biden_tweets")\
.setOutputCol("biden_document")

jb_use = UniversalSentenceEncoder.pretrained(lang="en")\
.setInputCols(["biden_document"])\
.setOutputCol("jb_sentence_embeddings")

jb_model_name = 'classifierdl_use_fakenews'
jb_document_classifier = ClassifierDLModel.pretrained(model_name)\
.setInputCols(['biden_document', 'jb_sentence_embeddings']).setOutputCol("jb_prediction")

jb_nlp_Pipeline = Pipeline( stages=[jb_documentAssembler,
            jb_use,
            jb_document_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


In [15]:
# Training (Pipeline)
dt_empty_df = spark.createDataFrame([['']]).toDF("trump_tweets") 
pipelineModel = nlp_Pipeline.fit(dt_empty_df)
dt_result_df = pipelineModel.transform(dt_df)

# Show Schema
dt_result_df.printSchema()

root
 |-- trump_tweets: string (nullable = true)
 |-- trump_document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- dt_sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: stri

In [18]:
# Training (Pipeline)
jb_empty_df = spark.createDataFrame([['']]).toDF("biden_tweets") 
jb_pipelineModel = jb_nlp_Pipeline.fit(jb_empty_df)
jb_result_df = jb_pipelineModel.transform(jb_df)

# Show Schema
jb_result_df.printSchema()

root
 |-- biden_tweets: string (nullable = true)
 |-- biden_document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- jb_sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: stri

In [16]:
import pyspark.sql.functions as F 
dt_result_df.select(F.explode(F.arrays_zip('dt_prediction.result', 'trump_document.result')).alias("cols"))\
.select(F.expr("cols['0']").alias("dt_prediction"), F.expr("cols['1']").alias("trump_document")).show(truncate=False)

+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|dt_prediction|trump_document                                                                                                                                                                                                                                                                                      |
+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|FAKE         |https://t.co/0LKTMOgk1T                                   

In [19]:
jb_result_df.select(F.explode(F.arrays_zip('jb_prediction.result', 'biden_document.result')).alias("cols"))\
.select(F.expr("cols['0']").alias("jb_prediction"), F.expr("cols['1']").alias("biden_document")).show(truncate=False)

+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|jb_prediction|biden_document                                                                                                                                                                                                                                                                                                |
+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|REAL         |I know it can be hard to kee